In [1]:
pip install soundfile

Note: you may need to restart the kernel to use updated packages.


In [2]:
import soundfile
print(soundfile.__version__)

0.12.1


In [3]:
import librosa
print(librosa.__version__)

0.10.1


In [9]:
import subprocess
import os
from pathlib import Path
import pandas as pd
from huggingface_hub import login
from datasets import load_dataset

In [10]:
ds = load_dataset('google/MusicCaps')

In [11]:
ds

DatasetDict({
    train: Dataset({
        features: ['ytid', 'start_s', 'end_s', 'audioset_positive_labels', 'aspect_list', 'caption', 'author_id', 'is_balanced_subset', 'is_audioset_eval'],
        num_rows: 5521
    })
})

In [12]:
train_data = ds['train']

# Converting to DataFrame
df = pd.DataFrame(train_data)


In [13]:
df

,ytid,start_s,end_s,audioset_positive_labels,aspect_list,caption,author_id,is_balanced_subset,is_audioset_eval
0,-0Gj8-vB1q4,30,40,"/m/0140xf,/m/02cjck,/m/04rlf","['low quality', 'sustained strings melody', 's...",The low quality recording features a ballad so...,4,False,True
1,-0SdAVK79lg,30,40,"/m/0155w,/m/01lyv,/m/0342h,/m/042v_gx,/m/04rlf...","['guitar song', 'piano backing', 'simple percu...",This song features an electric guitar as the m...,0,False,False
2,-0vPFx-wRRI,30,40,"/m/025_jnm,/m/04rlf","['amateur recording', 'finger snipping', 'male...",a male voice is singing a melody with changing...,6,False,True
3,-0xzrMun0Rs,30,40,"/m/01g90h,/m/04rlf","['backing track', 'jazzy', 'digital drums', 'p...",This song contains digital drums playing a sim...,6,False,True
4,-1LrH01Ei1w,30,40,"/m/02p0sh1,/m/04rlf","['rubab instrument', 'repetitive melody on dif...",This song features a rubber instrument being p...,0,False,False
...,...,...,...,...,...,...,...,...,...
5516,zw5dkiklbhE,15,25,"/m/01sm1g,/m/0l14md","['amateur recording', 'percussion', 'wooden bo...",This audio contains someone playing a wooden b...,6,False,False
5517,zwfo7wnXdjs,30,40,"/m/02p0sh1,/m/04rlf,/m/06j64v","['instrumental music', 'arabic music', 'genera...",The song is an instrumental. The song is mediu...,1,True,True
5518,zx_vcwOsDO4,50,60,"/m/01glhc,/m/02sgy,/m/0342h,/m/03lty,/m/04rlf,...","['instrumental', 'no voice', 'electric guitar'...",The rock music is purely instrumental and feat...,2,True,True
5519,zyXa2tdBTGc,30,40,"/m/04rlf,/t/dd00034","['instrumental music', 'gospel music', 'strong...",The song is an instrumental. The song is slow ...,1,False,False


In [14]:
import os

# Path to the folder containing the WAV files
wav_folder = 'music_data'  # Update this with your actual folder path

# Function to get the WAV file path based on YouTube ID
def get_wav_path(ytid):
    return os.path.join(wav_folder, f"{ytid}.wav")

# Create a list to store tuples of captions and WAV file paths
caption_wav_list = []

# Iterate through the rows of the original DataFrame
for index, row in df.iterrows():
    # Get the YouTube ID and caption
    ytid = row['ytid']
    caption = row['caption']
    
    # Get the WAV file path
    wav_path = get_wav_path(ytid)
    
    # Append the caption and WAV file path to the list
    caption_wav_list.append((caption, wav_path))

# Create a new DataFrame from the list
caption_wav_df = pd.DataFrame(caption_wav_list, columns=['caption', 'wav_path'])



In [9]:
caption_wav_df

,caption,wav_path
0,The low quality recording features a ballad so...,D:\TTM\music_data\-0Gj8-vB1q4.wav
1,This song features an electric guitar as the m...,D:\TTM\music_data\-0SdAVK79lg.wav
2,a male voice is singing a melody with changing...,D:\TTM\music_data\-0vPFx-wRRI.wav
3,This song contains digital drums playing a sim...,D:\TTM\music_data\-0xzrMun0Rs.wav
4,This song features a rubber instrument being p...,D:\TTM\music_data\-1LrH01Ei1w.wav
...,...,...
5516,This audio contains someone playing a wooden b...,D:\TTM\music_data\zw5dkiklbhE.wav
5517,The song is an instrumental. The song is mediu...,D:\TTM\music_data\zwfo7wnXdjs.wav
5518,The rock music is purely instrumental and feat...,D:\TTM\music_data\zx_vcwOsDO4.wav
5519,The song is an instrumental. The song is slow ...,D:\TTM\music_data\zyXa2tdBTGc.wav


In [10]:
caption_wav_df = caption_wav_df[:100]

In [11]:
caption_wav_df

,caption,wav_path
0,The low quality recording features a ballad so...,D:\TTM\music_data\-0Gj8-vB1q4.wav
1,This song features an electric guitar as the m...,D:\TTM\music_data\-0SdAVK79lg.wav
2,a male voice is singing a melody with changing...,D:\TTM\music_data\-0vPFx-wRRI.wav
3,This song contains digital drums playing a sim...,D:\TTM\music_data\-0xzrMun0Rs.wav
4,This song features a rubber instrument being p...,D:\TTM\music_data\-1LrH01Ei1w.wav
...,...,...
95,This is a rock music piece playing in the back...,D:\TTM\music_data\-taO6N-rxv4.wav
96,The low quality recording features a pop song ...,D:\TTM\music_data\-tmY1GEH3_Y.wav
97,This composition contains an upright bass play...,D:\TTM\music_data\-tpq_bzSKes.wav
98,An acoustic drum set is playing a 16th note rh...,D:\TTM\music_data\-uaTK8sa5Ms.wav


In [12]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Get list of stopwords
stop_words = set(stopwords.words('english'))

# Function for text preprocessing
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    return tokens

# Apply text preprocessing to captions in the DataFrame
caption_wav_df['processed_caption'] = caption_wav_df['caption'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tejas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\tejas\AppData\Local\Temp\ipykernel_21576\478400242.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  caption_wav_df['processed_caption'] = caption_wav_df['caption'].apply(preprocess_text)


In [13]:
caption_wav_df

,caption,wav_path,processed_caption
0,The low quality recording features a ballad so...,D:\TTM\music_data\-0Gj8-vB1q4.wav,"[low, quality, recording, features, ballad, so..."
1,This song features an electric guitar as the m...,D:\TTM\music_data\-0SdAVK79lg.wav,"[song, features, electric, guitar, main, instr..."
2,a male voice is singing a melody with changing...,D:\TTM\music_data\-0vPFx-wRRI.wav,"[male, voice, singing, melody, changing, tempo..."
3,This song contains digital drums playing a sim...,D:\TTM\music_data\-0xzrMun0Rs.wav,"[song, contains, digital, drums, playing, simp..."
4,This song features a rubber instrument being p...,D:\TTM\music_data\-1LrH01Ei1w.wav,"[song, features, rubber, instrument, played, s..."
...,...,...,...
95,This is a rock music piece playing in the back...,D:\TTM\music_data\-taO6N-rxv4.wav,"[rock, music, piece, playing, background, tuto..."
96,The low quality recording features a pop song ...,D:\TTM\music_data\-tmY1GEH3_Y.wav,"[low, quality, recording, features, pop, song,..."
97,This composition contains an upright bass play...,D:\TTM\music_data\-tpq_bzSKes.wav,"[composition, contains, upright, bass, playing..."
98,An acoustic drum set is playing a 16th note rh...,D:\TTM\music_data\-uaTK8sa5Ms.wav,"[acoustic, drum, set, playing, 16th, note, rhy..."


In [14]:
caption_wav_df['processed_caption']

0     [low, quality, recording, features, ballad, so...
1     [song, features, electric, guitar, main, instr...
2     [male, voice, singing, melody, changing, tempo...
3     [song, contains, digital, drums, playing, simp...
4     [song, features, rubber, instrument, played, s...
                            ...                        
95    [rock, music, piece, playing, background, tuto...
96    [low, quality, recording, features, pop, song,...
97    [composition, contains, upright, bass, playing...
98    [acoustic, drum, set, playing, 16th, note, rhy...
99    [soft, rock, song, features, passionate, femal...
Name: processed_caption, Length: 100, dtype: object

In [15]:
import os
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Function to convert WAV files to spectrograms and save images
def convert_to_spectrogram(wav_path, specto_folder):
    # Load the audio file
    try:
        y, sr = librosa.load(wav_path)
    except FileNotFoundError:
        print(f"File not found: {wav_path}")
        return None
    
    # Compute the spectrogram
    spectrogram = librosa.feature.melspectrogram(y=y, sr=sr)
    db_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    
    # Save the spectrogram image
    specto_path = os.path.join(specto_folder, '-'+os.path.basename(wav_path).replace('.wav', '.png'))
    plt.imsave(specto_path, db_spectrogram, cmap='viridis', format='png')
    
    return specto_path

# Create a folder for spectrograms if it doesn't exist
specto_folder = 'specto2'
if not os.path.exists(specto_folder):
    os.makedirs(specto_folder)

# Apply the function to each WAV file and store the paths
caption_wav_df['specto_path'] = caption_wav_df['wav_path'].apply(lambda x: convert_to_spectrogram(x, specto_folder))


C:\Users\tejas\AppData\Local\Temp\ipykernel_21576\1458204219.py:11: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(wav_path)
D:\Anaconda\Anaconda_Installation\envs\new_env\lib\site-packages\librosa\core\audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


File not found: D:\TTM\music_data\-sevczF5etI.wav


C:\Users\tejas\AppData\Local\Temp\ipykernel_21576\1458204219.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  caption_wav_df['specto_path'] = caption_wav_df['wav_path'].apply(lambda x: convert_to_spectrogram(x, specto_folder))


In [16]:
caption_wav_df

,caption,wav_path,processed_caption,specto_path
0,The low quality recording features a ballad so...,D:\TTM\music_data\-0Gj8-vB1q4.wav,"[low, quality, recording, features, ballad, so...",specto2\--0Gj8-vB1q4.png
1,This song features an electric guitar as the m...,D:\TTM\music_data\-0SdAVK79lg.wav,"[song, features, electric, guitar, main, instr...",specto2\--0SdAVK79lg.png
2,a male voice is singing a melody with changing...,D:\TTM\music_data\-0vPFx-wRRI.wav,"[male, voice, singing, melody, changing, tempo...",specto2\--0vPFx-wRRI.png
3,This song contains digital drums playing a sim...,D:\TTM\music_data\-0xzrMun0Rs.wav,"[song, contains, digital, drums, playing, simp...",specto2\--0xzrMun0Rs.png
4,This song features a rubber instrument being p...,D:\TTM\music_data\-1LrH01Ei1w.wav,"[song, features, rubber, instrument, played, s...",specto2\--1LrH01Ei1w.png
...,...,...,...,...
95,This is a rock music piece playing in the back...,D:\TTM\music_data\-taO6N-rxv4.wav,"[rock, music, piece, playing, background, tuto...",specto2\--taO6N-rxv4.png
96,The low quality recording features a pop song ...,D:\TTM\music_data\-tmY1GEH3_Y.wav,"[low, quality, recording, features, pop, song,...",specto2\--tmY1GEH3_Y.png
97,This composition contains an upright bass play...,D:\TTM\music_data\-tpq_bzSKes.wav,"[composition, contains, upright, bass, playing...",specto2\--tpq_bzSKes.png
98,An acoustic drum set is playing a 16th note rh...,D:\TTM\music_data\-uaTK8sa5Ms.wav,"[acoustic, drum, set, playing, 16th, note, rhy...",specto2\--uaTK8sa5Ms.png


In [17]:
caption_wav_df = caption_wav_df.dropna(subset=['specto_path'])

In [18]:
caption_wav_df

,caption,wav_path,processed_caption,specto_path
0,The low quality recording features a ballad so...,D:\TTM\music_data\-0Gj8-vB1q4.wav,"[low, quality, recording, features, ballad, so...",specto2\--0Gj8-vB1q4.png
1,This song features an electric guitar as the m...,D:\TTM\music_data\-0SdAVK79lg.wav,"[song, features, electric, guitar, main, instr...",specto2\--0SdAVK79lg.png
2,a male voice is singing a melody with changing...,D:\TTM\music_data\-0vPFx-wRRI.wav,"[male, voice, singing, melody, changing, tempo...",specto2\--0vPFx-wRRI.png
3,This song contains digital drums playing a sim...,D:\TTM\music_data\-0xzrMun0Rs.wav,"[song, contains, digital, drums, playing, simp...",specto2\--0xzrMun0Rs.png
4,This song features a rubber instrument being p...,D:\TTM\music_data\-1LrH01Ei1w.wav,"[song, features, rubber, instrument, played, s...",specto2\--1LrH01Ei1w.png
...,...,...,...,...
95,This is a rock music piece playing in the back...,D:\TTM\music_data\-taO6N-rxv4.wav,"[rock, music, piece, playing, background, tuto...",specto2\--taO6N-rxv4.png
96,The low quality recording features a pop song ...,D:\TTM\music_data\-tmY1GEH3_Y.wav,"[low, quality, recording, features, pop, song,...",specto2\--tmY1GEH3_Y.png
97,This composition contains an upright bass play...,D:\TTM\music_data\-tpq_bzSKes.wav,"[composition, contains, upright, bass, playing...",specto2\--tpq_bzSKes.png
98,An acoustic drum set is playing a 16th note rh...,D:\TTM\music_data\-uaTK8sa5Ms.wav,"[acoustic, drum, set, playing, 16th, note, rhy...",specto2\--uaTK8sa5Ms.png


In [19]:
from PIL import Image

# Load the image
image_path = r'D:\TTM\specto2\--0Gj8-vB1q4.png'
image = Image.open(image_path)

# Get the shape of the image
image_shape = image.size  # This returns width, height

print("Image shape:", image_shape)


Image shape: (861, 128)


In [20]:
max_caption_length = max(len(caption.split()) for caption in caption_wav_df['caption'])
input_shape = image_shape

In [21]:
max_caption_length

104

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the Generator model
def build_generator(latent_dim, caption_shape):
    noise_input = layers.Input(shape=(latent_dim,))
    caption_input = layers.Input(shape=caption_shape)
    
    # Concatenate noise and caption inputs
    concatenated_input = layers.Concatenate()([noise_input, caption_input])
    
    # Dense layers
    x = layers.Dense(128, activation='relu')(concatenated_input)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(512, activation='relu')(x)
    
    # Reshape
    x = layers.Reshape((861, 128))(x)  # Assuming input shape is (861, 128)
    
    # Output layer
    output = layers.Conv1DTranspose(1, kernel_size=7, strides=1, padding='same', activation='tanh')(x)
    
    # Define model
    model = models.Model(inputs=[noise_input, caption_input], outputs=output)
    return model

# Define the Discriminator model
def build_discriminator(input_shape, caption_shape):
    audio_input = layers.Input(shape=input_shape)
    caption_input = layers.Input(shape=caption_shape)
    
    # Concatenate the audio input and caption input
    concatenated_input = layers.Concatenate()([audio_input, caption_input])
    
    # Flatten the input
    x = layers.Flatten()(concatenated_input)
    
    # Dense layers
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    
    # Output layer
    output = layers.Dense(1, activation='sigmoid')(x)
    
    # Define model
    model = models.Model(inputs=[audio_input, caption_input], outputs=output)
    return model

# Define the size of the noise vector
latent_dim = 100

# Define the shapes of the caption inputs
caption_shape = (104,)  # Use the determined max_caption_length

# Define the input shape for the spectrogram data
input_shape = (861, 128)  # Use the determined image shape

# Build and compile the generator and discriminator models
generator = build_generator(latent_dim, caption_shape)
discriminator = build_discriminator(input_shape, caption_shape)

discriminator.compile(loss='binary_crossentropy', optimizer='adam')
discriminator.trainable = False

# Define the combined model (GAN)
noise_input = layers.Input(shape=(latent_dim,))
caption_input = layers.Input(shape=caption_shape)
generated_audio = generator([noise_input, caption_input])
validity = discriminator([generated_audio, caption_input])

gan = models.Model(inputs=[noise_input, caption_input], outputs=validity)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Display model architectures
generator.summary()
discriminator.summary()
gan.summary()


ValueError: The total size of the tensor must be unchanged. Received: input_shape=(512,), target_shape=(861, 128)

In [ ]:
;import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.utils import plot_model

# Define the size of the noise vector
latent_dim = 100

# Define the shapes of the caption inputs
caption_shape = (max_caption_length,)

# Define the training hyperparameters
epochs = 100
batch_size = 64
learning_rate = 0.0002
beta_1 = 0.5

# Define the binary crossentropy loss function
binary_crossentropy = BinaryCrossentropy(from_logits=True)

# Define the optimizer
optimizer = Adam(learning_rate, beta_1)

# Define the generator and discriminator models
generator = build_generator(latent_dim, caption_shape)
discriminator = build_discriminator(input_shape, caption_shape)

# Compile the discriminator
discriminator.compile(optimizer=optimizer, loss=binary_crossentropy)

# Define the combined model (GAN)
noise_input = layers.Input(shape=(latent_dim,))
caption_input = layers.Input(shape=caption_shape)
generated_audio = generator([noise_input, caption_input])
validity = discriminator([generated_audio, caption_input])

gan = models.Model(inputs=[noise_input, caption_input], outputs=validity)
gan.compile(optimizer=optimizer, loss=binary_crossentropy)

# Plot the generator and discriminator models
plot_model(generator, to_file='generator.png', show_shapes=True, show_layer_names=True)
plot_model(discriminator, to_file='discriminator.png', show_shapes=True, show_layer_names=True)

# Training loop
num_batches = len(caption_wav_df) // batch_size

for epoch in range(epochs):
    for batch_idx in range(num_batches):
        # Select a random batch of caption and audio samples
        batch_indices = np.random.choice(len(caption_wav_df), size=batch_size, replace=False)
        batch_captions = caption_wav_df['processed_caption'].iloc[batch_indices]
        batch_audio = load_audio_batch(caption_wav_df['wav_path'].iloc[batch_indices])
        
        # Generate a batch of random noise
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        
        # Generate fake audio samples using the generator
        generated_audio = generator.predict([noise, batch_captions])
        
        # Train the discriminator
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        
        d_loss_real = discriminator.train_on_batch([batch_audio, batch_captions], real_labels)
        d_loss_fake = discriminator.train_on_batch([generated_audio, batch_captions], fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train the generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss = gan.train_on_batch([noise, batch_captions], real_labels)
        
        # Print progress
        print(f"Epoch {epoch+1}/{epochs}, Batch {batch_idx+1}/{num_batches}, D Loss: {d_loss}, G Loss: {g_loss}")

    # Save models
    if (epoch + 1) % 10 == 0:
        generator.save(f'generator_epoch_{epoch+1}.h5')
        discriminator.save(f'discriminator_epoch_{epoch+1}.h5')


In [27]:
imgs = image_shape
capt = caption_shape
print(imgs)
print(capt)

(861, 128)
(104,)


In [28]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Concatenate, Dense
from tensorflow.keras.models import Model

# Define inputs
input1 = Input(shape=imgs)
input2 = Input(shape=capt)

# Concatenate the layers
concatenated_layer = Concatenate()([input1, input2])

# Output layer
output = Dense(1, activation='sigmoid')(concatenated_layer)

# Create model
model = Model(inputs=[input1, input2], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concatenation axis. Received: input_shape=[(None, 861, 128), (None, 104)]